<a href="https://colab.research.google.com/github/HemanthS3149/zoho_intern_work/blob/Week-4/Linear_Regression_and_Gradient_descent_using_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import torch

Gradient is the derivative of the loss

We can calculate the gradient wrt to the weights and biases if they have requires_grad set to True.

Gradient indicates the rate of change of loss

We can subtract from each weight element a small quantity proportional to the derivative of the loss wrt that element to reduce the loss slightly


**Linear regression using PyTorch builtins**

In [2]:
import torch.nn as nn

In [4]:
inputs = np.array([[73, 67, 43], #temp,rainfall,humidity
                   [91, 88, 64],
                   [87, 134, 58],
                   [102, 43, 37],
                   [69, 96, 70],
                   [74, 66, 43],
                   [91, 87, 65],
                   [88, 134, 59],
                   [101, 44, 37],
                   [68, 96, 71],
                   [73, 66, 44],
                   [92, 87, 64],
                   [87, 135, 57],
                   [103, 43, 36],
                   [68, 97, 70]],
                  dtype='float32')

targets=np.array([[56, 70],
                    [81, 101],
                    [119, 133], #apples and oranges
                    [22, 37],
                    [103, 119],
                    [57, 69],
                    [80, 102],
                    [118, 132],
                    [21, 38],
                    [104, 118],
                    [57, 69],
                    [82, 100],
                    [118, 134],
                    [20, 38],
                    [102, 120]],
                 dtype='float32')

inputs=torch.from_numpy(inputs) #converts numpy array into tensor
targets=torch.from_numpy(targets)

In [7]:
inputs,targets

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.],
         [102.,  43.,  37.],
         [ 69.,  96.,  70.],
         [ 74.,  66.,  43.],
         [ 91.,  87.,  65.],
         [ 88., 134.,  59.],
         [101.,  44.,  37.],
         [ 68.,  96.,  71.],
         [ 73.,  66.,  44.],
         [ 92.,  87.,  64.],
         [ 87., 135.,  57.],
         [103.,  43.,  36.],
         [ 68.,  97.,  70.]]),
 tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.],
         [ 22.,  37.],
         [103., 119.],
         [ 57.,  69.],
         [ 80., 102.],
         [118., 132.],
         [ 21.,  38.],
         [104., 118.],
         [ 57.,  69.],
         [ 82., 100.],
         [118., 134.],
         [ 20.,  38.],
         [102., 120.]]))

We use small training examples to illustrate how to work with large datasets in small batches

**Dataset and DataLoader**

A TensorDataset, allows access to rows from inputs and targets as tuples and provides standard APIs for working with many different types of datasets in PyTorch

In [8]:
from torch.utils.data import TensorDataset

In [9]:
#Define dataset
train_ds=TensorDataset(inputs,targets)
train_ds[0:3]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]),
 tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

**DataLoader** can split the data into batches of predifined size while training, and provides other utilities like shuffling and random sampling of data


In [11]:
from torch.utils.data import DataLoader

In [20]:
batch_size=5
train_dl=DataLoader(train_ds,batch_size,shuffle=True)#In each epoch we want different set of training data

In [21]:
for xb,yb in train_dl:
  print(xb)
  print(yb)
  break

tensor([[ 87., 135.,  57.],
        [ 68.,  96.,  71.],
        [ 73.,  66.,  44.],
        [ 69.,  96.,  70.],
        [ 68.,  97.,  70.]])
tensor([[118., 134.],
        [104., 118.],
        [ 57.,  69.],
        [103., 119.],
        [102., 120.]])


**nn.Linear** instead of initializing the w & b manually, we can define the model using nn.Linear


In [22]:
#Defining the model
model=nn.Linear(3,2)
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[ 0.0814, -0.4394,  0.2875],
        [-0.3187,  0.4343,  0.3237]], requires_grad=True)
Parameter containing:
tensor([0.3266, 0.5456], requires_grad=True)


In [23]:
#.parameters() method returns a list of all weights and bias matrices present
list(model.parameters())

[Parameter containing:
 tensor([[ 0.0814, -0.4394,  0.2875],
         [-0.3187,  0.4343,  0.3237]], requires_grad=True),
 Parameter containing:
 tensor([0.3266, 0.5456], requires_grad=True)]

In [24]:
#Generate predictions
preds=model(inputs)
preds

tensor([[-10.8076,  20.2923],
        [-12.5323,  30.4720],
        [-34.7937,  49.7816],
        [  0.3731,  -1.3158],
        [-16.1133,  42.9005],
        [-10.2869,  19.5393],
        [-11.8055,  30.3614],
        [-34.4248,  49.7866],
        [ -0.1477,  -0.5628],
        [-15.9072,  43.5429],
        [-10.0808,  20.1817],
        [-12.0115,  29.7190],
        [-35.5205,  49.8922],
        [  0.1670,  -1.9582],
        [-16.6341,  43.6535]], grad_fn=<AddmmBackward0>)

**Loss Function**: We can use the built in loss func mse_loss

In [26]:
import torch.nn.functional as F#contains many loss funcs and other utilities

In [27]:
loss_fn=F.mse_loss

In [28]:
loss=loss_fn(model(inputs),targets)
print(loss)

tensor(7290.9326, grad_fn=<MseLossBackward0>)


**Optimizer:** automatically manipulates the models weights and biases. (stochastic stands as in the samples are selected in random batches instead of a single group)

In [30]:
opt=torch.optim.SGD(model.parameters(),lr=1e-5) #Learning rate controls the amount by which the parameters are modified

**Train the model**

1.Generate predictions

2.Calculate the loss

3.Compute gradients wrt weights and biases

4.Adjusts the weights by subtracting a small quantity proportional to the gradient

5.Reset the gradients to zero

In [31]:
#Utility function to train the model
def fit(num_epochs,model,loss_fn,opt,train_dl):
  for epoch in range(num_epochs):
    for xb,yb in train_dl:
      pred=model(xb)
      loss=loss_fn(pred,yb)
      loss.backward() #compute gradients
      opt.step() #Update parameters using gradients
      opt.zero_grad()#reset the gradients to 0

    if(epoch+1)%10==0:
      print('Epoch [{}/{}],Loss:{:.4f}'.format(epoch+1,num_epochs,loss.item()))

In [32]:
fit(100,model,loss_fn,opt,train_dl)

Epoch [10/100],Loss:251.1195
Epoch [20/100],Loss:215.5683
Epoch [30/100],Loss:295.9462
Epoch [40/100],Loss:114.7477
Epoch [50/100],Loss:41.0195
Epoch [60/100],Loss:87.0533
Epoch [70/100],Loss:69.4483
Epoch [80/100],Loss:12.6375
Epoch [90/100],Loss:21.0829
Epoch [100/100],Loss:9.3110


In [33]:
preds=model(inputs)
preds


tensor([[ 58.3618,  71.1129],
        [ 82.9602,  98.9767],
        [113.5438, 135.9569],
        [ 28.9153,  40.5819],
        [ 98.8332, 114.1868],
        [ 57.3603,  70.0411],
        [ 82.9062,  98.7401],
        [113.9470, 136.3860],
        [ 29.9167,  41.6536],
        [ 99.7806, 115.0220],
        [ 58.3078,  70.8763],
        [ 81.9588,  97.9049],
        [113.5978, 136.1934],
        [ 27.9678,  39.7466],
        [ 99.8346, 115.2585]], grad_fn=<AddmmBackward0>)

In [34]:
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 57.,  69.],
        [ 80., 102.],
        [118., 132.],
        [ 21.,  38.],
        [104., 118.],
        [ 57.,  69.],
        [ 82., 100.],
        [118., 134.],
        [ 20.,  38.],
        [102., 120.]])

**Inference:** We have trained a reasonably good model to predict crop yeilds for apples and oranges by looking at the average temperature,rainfall and humidity in a region.

In [40]:
model(torch.tensor([[75,63,44.]]))#55 hectares for apples and 67 hectares for oranges
#model object which is an instance of nn.Linear is called with an input tensor
#this triggers the forward method of nn.Linear class which does input@weight +bias

tensor([[55.5757, 67.8641]], grad_fn=<AddmmBackward0>)